## ボールカウントの2018年データの集計

In [77]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import feather
pd.set_option('display.max_Columns', 100)

In [78]:
model_No = '30'

In [79]:
test_pitch_org = pd.read_feather('data/test_pitch.f')
print(test_pitch_org.shape)

predict_ball = pd.read_feather('intermediate/{}/ball_predict_2018.f'.format(model_No))
print(predict_ball.shape)

predict_course = pd.read_feather('intermediate/{}/course_predict_2018.f'.format(model_No))
print(predict_course.shape)

test_pitch_org = pd.concat([test_pitch_org, predict_ball], axis=1)
test_pitch_org = pd.concat([test_pitch_org, predict_course], axis=1)
print(test_pitch_org.shape)

(521650, 49)
(521650, 8)
(521650, 13)
(521650, 70)


In [80]:
test_pitch_org.head()

,データ内連番,年度,試合ID,試合内連番,試合内投球数,日付,時刻,ホームチームID,アウェイチームID,球場ID,球場名,試合種別詳細,イニング,表裏,イニング内打席数,打席内投球数,投手ID,投手チームID,投手投球左右,投手役割,投手登板順,投手試合内対戦打者数,投手試合内投球数,投手イニング内投球数,打者ID,打者チームID,打者打席左右,打者打順,打者守備位置,打者試合内打席数,プレイ前ホームチーム得点数,プレイ前アウェイチーム得点数,プレイ前アウト数,プレイ前ボール数,プレイ前ストライク数,プレイ前走者状況,一塁走者ID,二塁走者ID,三塁走者ID,捕手ID,一塁手ID,二塁手ID,三塁手ID,遊撃手ID,左翼手ID,中堅手ID,右翼手ID,成績対象投手ID,成績対象打者ID,predict_straight,predict_curve,predict_slider,predict_shoot,predict_fork,predict_changeup,predict_sinker,predict_cutball,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,predict_9,predict_10,predict_11,predict_12
0,0,2018,2018033001,1,1,2018-03-30,18:03:00,1,5,1,東京ドーム,セ・リーグ公式戦,1,表,1,1,1200041,1,右,先発,1,1,1,1,1500086,5,左,1,中,1,0,0,0,0,0,___,NaN,NaN,NaN,1300027,1400101,1600059,1200136,700003,1600184,600037,900397,1200041,1500086,0.327780,0.051419,0.120100,0.411858,0.003016,0.000178,0.000032,0.085617,0.063762,0.137020,0.068057,0.039368,0.059098,0.034506,0.056237,0.062408,0.038372,0.173545,0.090783,0.089161,0.087681
1,1,2018,2018033001,2,2,2018-03-30,18:03:39,1,5,1,東京ドーム,セ・リーグ公式戦,1,表,2,1,1200041,1,右,先発,1,2,2,2,400001,5,左,2,二,1,0,0,1,0,0,___,NaN,NaN,NaN,1300027,1400101,1600059,1200136,700003,1600184,600037,900397,1200041,400001,0.389656,0.081643,0.123798,0.354973,0.003965,0.000176,0.000013,0.045776,0.054210,0.141410,0.071047,0.032327,0.055780,0.030318,0.042724,0.064995,0.037974,0.162251,0.087444,0.108756,0.110764
2,2,2018,2018033001,3,3,2018-03-30,18:03:57,1,5,1,東京ドーム,セ・リーグ公式戦,1,表,2,2,1200041,1,右,先発,1,2,3,3,400001,5,左,2,二,1,0,0,1,0,1,___,NaN,NaN,NaN,1300027,1400101,1600059,1200136,700003,1600184,600037,900397,1200041,400001,0.350402,0.074194,0.133500,0.190600,0.178108,0.001092,0.000078,0.072026,0.035967,0.078084,0.052370,0.024460,0.039652,0.025470,0.041046,0.053942,0.028045,0.165346,0.119611,0.197080,0.138926
3,3,2018,2018033001,4,4,2018-03-30,18:04:40,1,5,1,東京ドーム,セ・リーグ公式戦,1,表,3,1,1200041,1,右,先発,1,3,4,4,400057,5,左,3,右,1,0,0,2,0,0,___,NaN,NaN,NaN,1300027,1400101,1600059,1200136,700003,1600184,600037,900397,1200041,400057,0.331941,0.103209,0.182694,0.306337,0.015705,0.000388,0.000030,0.059696,0.046990,0.109415,0.061691,0.027766,0.040484,0.024559,0.041230,0.062023,0.038416,0.185740,0.107148,0.132577,0.121962
4,4,2018,2018033001,5,5,2018-03-30,18:04:56,1,5,1,東京ドーム,セ・リーグ公式戦,1,表,3,2,1200041,1,右,先発,1,3,5,5,400057,5,左,3,右,1,0,0,2,1,0,___,NaN,NaN,NaN,1300027,1400101,1600059,1200136,700003,1600184,600037,900397,1200041,400057,0.164242,0.055456,0.289854,0.392830,0.008055,0.000407,0.000034,0.089121,0.059695,0.117593,0.067742,0.035094,0.049109,0.038872,0.049572,0.075280,0.046009,0.154545,0.074437,0.116909,0.115143


In [81]:
sample_No = 1
div = 66265

In [82]:
test_query = 'index <= {}'.format(div)
test_pitch = test_pitch_org.query(test_query).copy()
print(test_pitch.shape)

(66266, 70)


In [83]:
test_pitch['ball_cnt'] = test_pitch['プレイ前ストライク数'].astype(str) + '-' + test_pitch['プレイ前ボール数'].astype(str)

In [84]:
test_pitch.replace('左', 'L', inplace=True)
test_pitch.replace('右', 'R', inplace=True)
test_pitch['pit_bat'] = test_pitch['投手投球左右'] + '_' + test_pitch['打者打席左右']

### 球種

In [85]:
pitch_ball = test_pitch[['ball_cnt','pit_bat',
                                'predict_straight','predict_curve','predict_slider','predict_shoot',
                                'predict_fork','predict_changeup','predict_sinker','predict_cutball']]
groupby_pit = pitch_ball.groupby(['ball_cnt', 'pit_bat']).sum()
pitch_ball = groupby_pit.reset_index()
pitch_ball['ball_total'] = (pitch_ball['predict_straight'] + pitch_ball['predict_curve'] + pitch_ball['predict_slider'] + pitch_ball['predict_shoot']
                            + pitch_ball['predict_fork'] + pitch_ball['predict_changeup'] + pitch_ball['predict_sinker'] + pitch_ball['predict_cutball'])


In [86]:
print(pitch_ball.shape)
pitch_ball.head()

(48, 11)


,ball_cnt,pit_bat,predict_straight,predict_curve,predict_slider,predict_shoot,predict_fork,predict_changeup,predict_sinker,predict_cutball,ball_total
0,0-0,L_L,933.547597,129.767929,473.221490,49.737556,12.444900,31.733976,5.579050,51.967501,1688.0
1,0-0,L_R,1219.345208,244.067311,431.845740,161.279559,56.096574,255.454101,24.940041,82.971466,2476.0
2,0-0,R_L,3068.411682,672.181326,935.700630,471.603645,371.002830,251.891168,66.684942,274.523777,6112.0
3,0-0,R_R,3302.793714,654.424692,1494.896981,328.143642,223.601529,87.637388,25.213524,369.288532,6486.0
4,0-1,L_L,387.837000,32.857650,204.002812,31.096832,4.768640,14.089606,2.980436,29.367024,707.0


In [87]:
pitch_ball['bc_straight'] = pitch_ball['predict_straight']/pitch_ball['ball_total']
pitch_ball['bc_curve'] = pitch_ball['predict_curve']/pitch_ball['ball_total']
pitch_ball['bc_slider'] = pitch_ball['predict_slider']/pitch_ball['ball_total']
pitch_ball['bc_shoot'] = pitch_ball['predict_shoot']/pitch_ball['ball_total']
pitch_ball['bc_fork'] = pitch_ball['predict_fork']/pitch_ball['ball_total']
pitch_ball['bc_changeup'] = pitch_ball['predict_changeup']/pitch_ball['ball_total']
pitch_ball['bc_sinker'] = pitch_ball['predict_sinker']/pitch_ball['ball_total']
pitch_ball['bc_cutball'] = pitch_ball['predict_cutball']/pitch_ball['ball_total']

In [88]:
pitch_ball.drop(columns=[
    'ball_total',
    'predict_straight', 'predict_curve', 'predict_slider', 'predict_shoot',
    'predict_fork', 'predict_changeup', 'predict_sinker', 'predict_cutball'
], inplace=True)

In [89]:
print(pitch_ball.shape)
pitch_ball.head(10)

(48, 10)


,ball_cnt,pit_bat,bc_straight,bc_curve,bc_slider,bc_shoot,bc_fork,bc_changeup,bc_sinker,bc_cutball
0,0-0,L_L,0.553050,0.076877,0.280344,0.029465,0.007373,0.018800,0.003305,0.030786
1,0-0,L_R,0.492466,0.098573,0.174413,0.065137,0.022656,0.103172,0.010073,0.033510
2,0-0,R_L,0.502031,0.109977,0.153092,0.077160,0.060701,0.041213,0.010910,0.044916
3,0-0,R_R,0.509219,0.100898,0.230481,0.050593,0.034474,0.013512,0.003887,0.056936
4,0-1,L_L,0.548567,0.046475,0.288547,0.043984,0.006745,0.019929,0.004216,0.041538
5,0-1,L_R,0.484357,0.054299,0.191300,0.089066,0.019709,0.096554,0.015448,0.049266
6,0-1,R_L,0.517701,0.057242,0.177438,0.090591,0.046829,0.036628,0.016572,0.057000
7,0-1,R_R,0.499286,0.071060,0.240195,0.068926,0.032330,0.017065,0.005987,0.065150
8,0-2,L_L,0.693547,0.016277,0.204492,0.029124,0.003326,0.009118,0.004750,0.039365
9,0-2,L_R,0.586044,0.021439,0.156955,0.090577,0.016342,0.065529,0.013882,0.049231


### コース

In [90]:
pitch_course = test_pitch[['ball_cnt','pit_bat',
                            'predict_0','predict_1','predict_2','predict_3','predict_4','predict_5',
                            'predict_6','predict_7','predict_8','predict_9','predict_10','predict_11','predict_12']]
groupby_course = pitch_course.groupby(['ball_cnt','pit_bat']).sum()
pitch_course = groupby_course.reset_index(inplace=False)
pitch_course['total'] = (pitch_course['predict_0'] + pitch_course['predict_1'] + pitch_course['predict_2'] + pitch_course['predict_3'] + 
pitch_course['predict_4'] + pitch_course['predict_5'] + pitch_course['predict_6'] + pitch_course['predict_7'] + 
pitch_course['predict_8'] + pitch_course['predict_9'] + pitch_course['predict_10'] + pitch_course['predict_11'] + pitch_course['predict_12'])


In [91]:
pitch_course['bc_course_0'] = pitch_course['predict_0']/pitch_course['total']
pitch_course['bc_course_1'] = pitch_course['predict_1']/pitch_course['total']
pitch_course['bc_course_2'] = pitch_course['predict_2']/pitch_course['total']
pitch_course['bc_course_3'] = pitch_course['predict_3']/pitch_course['total']
pitch_course['bc_course_4'] = pitch_course['predict_4']/pitch_course['total']
pitch_course['bc_course_5'] = pitch_course['predict_5']/pitch_course['total']
pitch_course['bc_course_6'] = pitch_course['predict_6']/pitch_course['total']
pitch_course['bc_course_7'] = pitch_course['predict_7']/pitch_course['total']
pitch_course['bc_course_8'] = pitch_course['predict_8']/pitch_course['total']
pitch_course['bc_course_9'] = pitch_course['predict_9']/pitch_course['total']
pitch_course['bc_course_10'] = pitch_course['predict_10']/pitch_course['total']
pitch_course['bc_course_11'] = pitch_course['predict_11']/pitch_course['total']
pitch_course['bc_course_12'] = pitch_course['predict_12']/pitch_course['total']

In [92]:
pitch_course.drop(columns=[
    'predict_0', 'predict_1', 'predict_2', 'predict_3', 'predict_4', 'predict_5', 
    'predict_6', 'predict_7', 'predict_8', 'predict_9', 'predict_10', 'predict_11', 'predict_12'], inplace=True)

### コースの種類

In [93]:
pitch_course['bc_high_str'] = pitch_course['bc_course00'] + pitch_course['bc_course03'] + pitch_course['bc_course06'] 
pitch_course['bc_high_ball'] = pitch_course['bc_course09'] + pitch_course['bc_course10'] 
pitch_course['bc_mid_str'] = pitch_course['bc_course01'] + pitch_course['bc_course04'] + pitch_course['bc_course07'] 
pitch_course['bc_low_str'] = pitch_course['bc_course02'] + pitch_course['bc_course05'] + pitch_course['bc_course08'] 
pitch_course['bc_low_ball'] = pitch_course['bc_course11'] + pitch_course['bc_course12'] 

In [94]:
pitch_course['bc_left_str'] = pitch_course['bc_course00'] + pitch_course['bc_course01'] + pitch_course['bc_course02'] 
pitch_course['bc_left_ball'] = pitch_course['bc_course09'] + pitch_course['bc_course11'] 
pitch_course['bc_center_str'] = pitch_course['bc_course03'] + pitch_course['bc_course04'] + pitch_course['bc_course05'] 
pitch_course['bc_right_str'] = pitch_course['bc_course06'] + pitch_course['bc_course07'] + pitch_course['bc_course08'] 
pitch_course['bc_right_ball'] = pitch_course['bc_course10'] + pitch_course['bc_course12'] 

In [95]:
pitch_course.drop(columns=[
    'bc_course00', 'bc_course01', 'bc_course02', 'bc_course03', 'bc_course04', 'bc_course05', 
    'bc_course06', 'bc_course07', 'bc_course08', 'bc_course09', 'bc_course10', 'bc_course11', 'bc_course12'], inplace=True)

In [96]:
print(pitch_course.shape)
pitch_course.head()

(48, 13)


,ball_cnt,pit_bat,total,bc_high_str,bc_high_ball,bc_mid_str,bc_low_str,bc_low_ball,bc_left_str,bc_left_ball,bc_center_str,bc_right_str,bc_right_ball
0,0-0,L_L,1688.0,0.145223,0.209276,0.218786,0.166426,0.260289,0.293198,0.316705,0.148853,0.088384,0.152860
1,0-0,L_R,2476.0,0.139811,0.223414,0.196481,0.155713,0.284580,0.156929,0.245977,0.157098,0.177979,0.262017
2,0-0,R_L,6112.0,0.140803,0.240351,0.204691,0.158136,0.256019,0.241295,0.300918,0.151381,0.110955,0.195452
3,0-0,R_R,6486.0,0.155692,0.210660,0.207001,0.143113,0.283533,0.104907,0.158701,0.161799,0.239100,0.335493
4,0-1,L_L,707.0,0.157000,0.171869,0.227561,0.194148,0.249422,0.314662,0.280473,0.164360,0.099687,0.140818


### マージ

In [97]:
pitch_ball = pitch_ball.merge(pitch_course, on=['ball_cnt', 'pit_bat'], how='left')
print(pitch_ball.shape)
pitch_ball.head(10)

(48, 21)


,ball_cnt,pit_bat,bc_straight,bc_curve,bc_slider,bc_shoot,bc_fork,bc_changeup,bc_sinker,bc_cutball,total,bc_high_str,bc_high_ball,bc_mid_str,bc_low_str,bc_low_ball,bc_left_str,bc_left_ball,bc_center_str,bc_right_str,bc_right_ball
0,0-0,L_L,0.553050,0.076877,0.280344,0.029465,0.007373,0.018800,0.003305,0.030786,1688.0,0.145223,0.209276,0.218786,0.166426,0.260289,0.293198,0.316705,0.148853,0.088384,0.152860
1,0-0,L_R,0.492466,0.098573,0.174413,0.065137,0.022656,0.103172,0.010073,0.033510,2476.0,0.139811,0.223414,0.196481,0.155713,0.284580,0.156929,0.245977,0.157098,0.177979,0.262017
2,0-0,R_L,0.502031,0.109977,0.153092,0.077160,0.060701,0.041213,0.010910,0.044916,6112.0,0.140803,0.240351,0.204691,0.158136,0.256019,0.241295,0.300918,0.151381,0.110955,0.195452
3,0-0,R_R,0.509219,0.100898,0.230481,0.050593,0.034474,0.013512,0.003887,0.056936,6486.0,0.155692,0.210660,0.207001,0.143113,0.283533,0.104907,0.158701,0.161799,0.239100,0.335493
4,0-1,L_L,0.548567,0.046475,0.288547,0.043984,0.006745,0.019929,0.004216,0.041538,707.0,0.157000,0.171869,0.227561,0.194148,0.249422,0.314662,0.280473,0.164360,0.099687,0.140818
5,0-1,L_R,0.484357,0.054299,0.191300,0.089066,0.019709,0.096554,0.015448,0.049266,1053.0,0.151270,0.184652,0.212370,0.179594,0.272115,0.179321,0.230150,0.174707,0.189206,0.226616
6,0-1,R_L,0.517701,0.057242,0.177438,0.090591,0.046829,0.036628,0.016572,0.057000,2640.0,0.160143,0.190337,0.233197,0.181256,0.235067,0.270075,0.246034,0.180906,0.123615,0.179370
7,0-1,R_R,0.499286,0.071060,0.240195,0.068926,0.032330,0.017065,0.005987,0.065150,2672.0,0.160866,0.174192,0.221501,0.173595,0.269846,0.125598,0.151158,0.187510,0.242854,0.292880
8,0-2,L_L,0.693547,0.016277,0.204492,0.029124,0.003326,0.009118,0.004750,0.039365,216.0,0.183811,0.174844,0.247812,0.196863,0.196670,0.331376,0.243769,0.195792,0.101318,0.127744
9,0-2,L_R,0.586044,0.021439,0.156955,0.090577,0.016342,0.065529,0.013882,0.049231,346.0,0.175792,0.189965,0.232606,0.189694,0.211942,0.180432,0.185662,0.209345,0.208316,0.216245


In [98]:
#ball_cnt_all.to_feather(OUTPUT)

In [99]:
test_ball = pd.read_feather('intermediate/pitch/ball_2018_{}.f'.format(1))
test_ball.head()

,ball_cnt,pit_bat,bc_straight,bc_curve,bc_slider,bc_shoot,bc_fork,bc_changeup,bc_sinker,bc_cutball,total,bc_course_0,bc_course_1,bc_course_2,bc_course_3,bc_course_4,bc_course_5,bc_course_6,bc_course_7,bc_course_8,bc_course_9,bc_course_10,bc_course_11,bc_course_12,bc_high_str,bc_high_ball,bc_mid_str,bc_low_str,bc_low_ball,bc_left_str,bc_left_ball,bc_center_str,bc_right_str,bc_right_ball
0,0-0,L_L,0.553050,0.076877,0.280344,0.029465,0.007373,0.018800,0.003305,0.030786,1688.0,0.069175,0.117974,0.106048,0.047237,0.064152,0.037464,0.028810,0.036661,0.022913,0.102350,0.106925,0.214354,0.045935,0.145223,0.209276,0.218786,0.166426,0.260289,0.293198,0.316705,0.148853,0.088384,0.152860
1,0-0,L_R,0.492466,0.098573,0.174413,0.065137,0.022656,0.103172,0.010073,0.033510,2476.0,0.039800,0.062810,0.054318,0.048704,0.061724,0.046670,0.051307,0.071947,0.054725,0.079309,0.144105,0.166668,0.117912,0.139811,0.223414,0.196481,0.155713,0.284580,0.156929,0.245977,0.157098,0.177979,0.262017
2,0-0,R_L,0.502031,0.109977,0.153092,0.077160,0.060701,0.041213,0.010910,0.044916,6112.0,0.068280,0.095701,0.077313,0.042190,0.062749,0.046442,0.030332,0.046241,0.034381,0.177678,0.062673,0.123240,0.132779,0.140803,0.240351,0.204691,0.158136,0.256019,0.241295,0.300918,0.151381,0.110955,0.195452
3,0-0,R_R,0.509219,0.100898,0.230481,0.050593,0.034474,0.013512,0.003887,0.056936,6486.0,0.035690,0.043677,0.025540,0.054885,0.065905,0.041009,0.065116,0.097420,0.076564,0.105218,0.105443,0.053483,0.230051,0.155692,0.210660,0.207001,0.143113,0.283533,0.104907,0.158701,0.161799,0.239100,0.335493
4,0-1,L_L,0.548567,0.046475,0.288547,0.043984,0.006745,0.019929,0.004216,0.041538,707.0,0.071877,0.125082,0.117703,0.050695,0.064304,0.049362,0.034428,0.038175,0.027083,0.082533,0.089336,0.197941,0.051482,0.157000,0.171869,0.227561,0.194148,0.249422,0.314662,0.280473,0.164360,0.099687,0.140818
